In [1]:
#设置显示全部行，不省略
import pandas as pd
pd.set_option('display.max_rows',None)
#设置显示全部列，不省略
pd.set_option('display.max_columns',None)
import akshare as ak
import warnings
warnings.filterwarnings("ignore")

In [2]:
import akshare as ak
import talib
import pandas as pd
import numpy as np
import time


def get_data(code):
    df = ak.stock_zh_a_hist(symbol=code, start_date='20220301',end_date='20240326', adjust="qfq").iloc[:, :6]
    df.columns = ['date','open','close','high','low','volume',]
    df.index = pd.to_datetime(df.date)
    df["openinterest"] = 0
    #df = df[["open", "high", "low", "close", "volume", "openinterest"]]
    return df

data = get_data("002230")

data['ma5'] = talib.MA(data['close'], timeperiod=5)
data['ma10'] = talib.MA(data['close'], timeperiod=10)
data['ma20'] = talib.MA(data['close'], timeperiod=20)
data['bias1'] = (data['close'] - data['ma5']) / data['ma5'] * 100
data['bias2'] = (data['close'] - data['ma10']) / data['ma10'] * 100
data['bias3'] = (data['close'] - data['ma20']) / data['ma20'] * 100
data['upper'], data['middle'], data['lower'] = talib.BBANDS(data['close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['k'], data['d'] = talib.STOCH(data['high'], data['low'], data['close'], fastk_period=9, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
data['rsi'] = talib.RSI(data['close'], timeperiod=14)

condition1 = (data['ma5'] > data['ma10']) & (data['ma5'] > data['ma20']) & (data['bias1'] > data['bias2']) & (data['bias1'] > data['bias3'])
fcondition1 = (data['ma5'] <= data['ma10']) & (data['ma5'] <= data['ma20']) & (data['bias1'] <= data['bias2']) & (data['bias1'] <= data['bias3'])


condition2 = (data['close'] < data['lower'])
fcondition2 = (data['close'] >= data['lower'])


condition3 = (data['k'] > data['d']) & (data['k'].shift() < data['d'].shift())
fcondition3 = (data['k'] <= data['d']) & (data['k'].shift() >= data['d'].shift())


condition4 = (data['rsi'] > 80) | (data['rsi'] < 20)
fcondition4 = (data['rsi'] <= 80) | (data['rsi'] >= 20)

data['signals'] = 0.2 * condition1.astype(int)+0.3 * condition2.astype(int)+0.2 * condition3.astype(int)+0.3 * condition4.astype(int)
data['fsignals'] = 0.2 * fcondition1.astype(int)+0.3 * fcondition2.astype(int)+0.2 * fcondition3.astype(int)+0.3 * fcondition4.astype(int)
data['b'] = data['signals'].apply(lambda x: 1 if x> 0 else 0)
data['s'] = data['fsignals'].apply(lambda x: 1 if x> 0 else 0)


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from lxml import etree
import pymysql
# https://blog.csdn.net/xiongmaohenku/article/details/120832138
def get_zjlx_data(code):
    conn=pymysql.connect(host='localhost',user='root',password='199037',database='hongya',port=3306,charset='utf8')
    cursor = conn.cursor()
    cursor.execute("drop table if exists 资金流向表")
    sql = """
            create table 资金流向表(
            代码 VARCHAR(255),
            日期 VARCHAR(255),
            收盘价 VARCHAR(255),
            涨跌幅 VARCHAR(255),
            主力净流入净额 VARCHAR(255),
            主力净流入净占比 VARCHAR(255),
            超大单净流入净额 VARCHAR(255),
            超大单净流入净占比 VARCHAR(255),
            大单净流入净额 VARCHAR(255),
            大单净流入净占比 VARCHAR(255),
            中单净流入净额 VARCHAR(255),
            中单净流入净占比 VARCHAR(255),
            小单净流入净额 VARCHAR(255),
            小单净流入净占比 VARCHAR(255)
            )
        """
    cursor.execute(sql)
    sql = """insert into 资金流向表 values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""


    option = Options()
    option.add_argument('headless')  # 设置option，不弹出浏览器
    #driver = webdriver.Chrome(options=option)
    driver = webdriver.Edge(options=option)
    driver.get('https://data.eastmoney.com/zjlx/{}.html'.format(code))
    page = driver.page_source # 获取页面源码

    res = etree.HTML(page) # 解析源码
    tr_list = res.xpath('//*[@id="table_ls"]/table/tbody/tr') # 使用xpath提取数据
    for i in range(len(tr_list)):
        data = tr_list[i].xpath('.//text()')
        data.insert(0, code)
        cursor.execute(sql, data)
    conn.commit()
    cursor.close()
    conn.close()

    driver.quit()
# get_zjlx_data("002230")# 47-48.5
# get_zjlx_data("000977")#40.5
# get_zjlx_data("001872")#47-47.5
get_zjlx_data("000612")#47-47.5
#https://blog.csdn.net/qq_43125235/article/details/125601564
#https://blog.csdn.net/m0_46540018/article/details/120813974

ModuleNotFoundError: No module named 'selenium'

In [ ]:
#https://www.dongao.com/acca/bkjy/202205253794932.shtml
#https://www.cnblogs.com/waizaowang/p/15695492.html
ACCOUNTS_PAYABLE 应付账款
ACCOUNTS_RECE 应收账款
ACCRUED_EXPENSE 应计费用
ADVANCE_RECEIVABLES 预收款

AMORTIZE_COST_FINASSET 以摊余成本计量的金融资产

APPOINT_FVTPL_FINASSET 指定以公允价值计量且其变动计入当期损益的金融资产 其中:指定以公允价值计量且其变动计入当期损益的金融资
APPOINT_FVTPL_FINLIAB 指定以公允价值计量且其变动计入当期损益的金融负债 其中:指定以公允价值计量且其变动计入当期损益的金融负
BOND_PAYABLE 资产平衡项目
BORROW_FUND 拆入资金
BUY_RESALE_FINASSET 买入返售金融资产
CAPITAL_RESERVE
CIP 在建工程
CONSUMPTIVE_BIOLOGICAL_ASSET
CONTRACT_ASSET
CONTRACT_LIAB
CONVERT_DIFF
TRADE_FINASSET 交易性金融资产
TRADE_FINASSET_NOTFVTPL
TRADE_FINLIAB 交易性金融负债
TRADE_FINLIAB_NOTFVTPL 交易性金融负债
TREASURY_SHARES 库存股
UNASSIGN_RPOFIT 未分配利润
UNCONFIRM_INVEST_LOSS 未确认投资损失
USERIGHT_ASSET 使用权资产
字数限制，有些实在难懂，只能从不同年报中看到对应的解释

['0、secucode', '600519.SH', '股票代码',
'1、security_code', '600519', '股票代码',
'2、security_name_abbr', '贵州茅台', '股票名称',
'3、org_code', '10002602', '机构代码',
'4、org_type', '通用', '机构类型',
'5、report_date', '2022-12-31 00:00:00', '报告时间',
'6、report_type', '年报', '报告类型',
'7、report_date_name', '2022年报', '报告时期',
'8、security_type_code', '058001001', '安全码',
'9、notice_date', '2023-03-31 00:00:00', '公告时间',
'10、update_date', '2023-04-26 00:00:00', '披露时间',
'11、currency', 'CNY','货币',
'12、accept_deposit_interbank', '12874043355.42', '',
'13、accounts_payable', '2408371053.69', '应付账款',
'14、accounts_rece', '20937144.0', '应收账款',
'15、accrued_expense', 'nan', '',
'16、advance_receivables', 'nan', '预收款项',
'17、agent_trade_security', 'None', '',
'18、agent_underwrite_security', 'None', '',
'19、amortize_cost_finasset', 'None', '',
'20、amortize_cost_finliab', 'None', '',
'21、amortize_cost_ncfinasset', 'None', '',
'22、amortize_cost_ncfinliab', 'None', '',
'23、appoint_fvtpl_finasset', 'None', '',
'24、appoint_fvtpl_finliab', 'None', '',
'25、asset_balance', '0.0','',
'26、asset_other', 'nan', '',
'27、assign_cash_dividend', 'None', '',
'28、available_sale_finasset', 'nan','',
'29、bond_payable', 'None', '应付债券',
'30、borrow_fund', 'None', '',
'31、buy_resale_finasset', 'None', '',
'32、capital_reserve', '1374964415.72', '资本公积',
'33、cip', '2208329892.95', '在建工程',
'34、consumptive_biological_asset','None', '',
'35、contract_asset', 'None', '合同资产',
'36、contract_liab', '15471920924.98', '合同负债',
'37、convert_diff','None', '',
'38、creditor_invest', '380685319.09', '债权投资',
'39、current_asset_balance', '0.0', '',
'40、current_asset_other', 'nan', '',
'41、current_liab_balance', '0.0', '',
'42、current_liab_other', 'nan', '',
'43、defer_income', 'None', '递延收益',
'44、defer_income_1year', 'None', '',
'45、defer_tax_asset', '3458931368.11', '递延所得税资产',
'46、defer_tax_liab', 'nan', '递延所得税负债',
'47、derive_finasset', 'None', '衍生金融资产',
'48、derive_finliab', 'None', '衍生金融负债',
'49、develop_expense', '190536632.6', '开发支出',
'50、div_holdsale_asset', 'None', '',
'51、div_holdsale_liab', 'None','',
'52、dividend_payable', 'nan', '应付股利',
'53、dividend_rece', 'None', '',
'54、equity_balance', '0.0', '',
'55、equity_other', 'nan', '',
'56、export_refund_rece', 'None', '',
'57、fee_commission_payable', 'None', '',
'58、fin_fund', 'None', '',
'59、finance_rece', 'None', '应收款项融资',
'60、fixed_asset', '19742622547.86', '固定资产',
'61、fixed_asset_disposal', 'nan', '',
'62、fvtoci_finasset', 'None', '',
'63、fvtoci_ncfinasset', 'None', '',
'64、fvtpl_finasset', 'None', '',
'65、fvtpl_finliab', 'None', '',
'66、general_risk_reserve', '1061529724.0','一般风险准备',
'67、goodwill', 'None', '商誉',
'68、hold_maturity_invest', 'nan', '',
'69、holdsale_asset', 'None', '持有待售资产',
'70、holdsale_liab', 'None', '',
'71、insurance_contract_reserve', 'None', '',
'72、intangible_asset','7083177226.45', '无形资产',
'73、interest_payable', 'nan', '',
'74、interest_rece', 'nan', '',
'75、internal_payable','None', '',
'76、internal_rece', 'None', '',
'77、inventory', '38824374236.24', '存货',
'78、invest_realestate','5335046.99', '投资性房地产',
'79、lease_liab', '334447942.79', '租赁负债',
'80、lend_fund', '116172711554.59', '拆出资金',
'81、liab_balance', '0.0', '',
'82、liab_equity_balance', 'nan', '',
'83、liab_equity_other', 'None', '',

SyntaxError: invalid character '，' (U+FF0C) (2239601306.py, line 29)